In [ ]:
import time
import random
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
UP, DOWN = 1, -1

class Person: 

    def __init__(self, target_floor, spawn_timepoint) -> None:
        self.target_floor = target_floor
        self.spawn_timepoint = spawn_timepoint

class Elevator:

    def __init__(self, capacity) -> None:
        self.capacity = capacity
        self.people = []
        self.current_floor = 1
        self.current_direction = UP
    
class Building:

    def __init__(self, num_floors) -> None:
        self.num_floors = num_floors
        self.lounge = []
        for _ in range(num_floors):
            self.lounge.append([])
        self.elevator : Elevator = Elevator(5)
    
    def step(self):
        unloaded_people = [p for p in self.elevator.people if p.target_floor == self.elevator.current_floor]
        self.elevator.people = [p for p in self.elevator.people if p.target_floor != self.elevator.current_floor]
        while len(self.elevator.people)<self.elevator.capacity:
            try:
                self.elevator.people.append(self.lounge[self.elevator.current_floor-1].pop())
            except IndexError:
                break
        self.elevator.current_floor += self.elevator.current_direction
        if self.elevator.current_floor in [1, self.num_floors]:
            self.elevator.current_direction *= -1
        return unloaded_people

    def add_person(self, p :Person, floor):
        self.lounge[floor-1].append(p)

    def __str__(self) -> str:
        reprr=""
        for i, l in enumerate(self.lounge[::-1]):
            reprr+="|"+ ("<" if (self.num_floors-i)==self.elevator.current_floor else "" ) + str(self.num_floors-i) +"|\t" + "o "*len(l) + "\n"
        reprr += f"people on elevator: {len(self.elevator.people)}\n"

        return reprr


In [ ]:
def simulate(nmax = 1, _p=0.1):
    b = Building(10)
    avg_wait_times = []
    wait_times = []
    for tstep in range(1000):
        if tstep<901:
            for _ in range(nmax):
                p = random.random()
                if p<_p:
                    cfloor = random.randint(1, 10)
                    tfloor = random.randint(1,10)
                    while tfloor!=cfloor:
                        tfloor = random.randint(1,10)
                    b.add_person(Person(tfloor, tstep), cfloor)
        ps = b.step()
        wait_times += [tstep - p.spawn_timepoint for p in ps]
        if (len(wait_times)>0):
            avg_wait_time = sum(wait_times) / len(wait_times)
        else:
            avg_wait_time = 0
        avg_wait_times.append(avg_wait_time)
        # with open("building.txt", "w") as f:
            # f.write(str(b))
            # f.write(f"{avg_wait_time}")
        # time.sleep(0.5)
    return avg_wait_times 

In [ ]:
ss = np.array([simulate(1) for _ in range(100)])
plt.plot(ss.mean(axis=0))
plt.fill_between(np.arange(0,1000,1),np.percentile(ss,67,axis=0), np.percentile(ss,33,axis=0), alpha=0.1)

ss = np.array([simulate(5) for _ in range(100)])
plt.plot(ss.mean(axis=0))
plt.fill_between(np.arange(0,1000,1),np.percentile(ss,67,axis=0), np.percentile(ss,33,axis=0), alpha=0.1)

ss = np.array([simulate(3) for _ in range(100)])
plt.plot(ss.mean(axis=0))
plt.fill_between(np.arange(0,1000,1),np.percentile(ss,67,axis=0), np.percentile(ss,33,axis=0), alpha=0.1)

In [ ]:
ps = [0.005*i for i in range(1,120)]
ys = []

for p in ps:
    ss = np.array([simulate(1, p) for _ in range(200)])
    ys.append(ss[:,-1].mean())

In [ ]:
plt.scatter(ps, ys)